<a href="https://colab.research.google.com/github/mtsilimos/Codesnippets_Sentence-Embeddings-for-the-Classification-of-Explicit-Fine-grained-Discourse-Relations/blob/main/GridSearch_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/finalcorpus.csv', ';', encoding= 'unicode_escape')

df['text'].head()


<ipython-input-63-e599d8470047>:3: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv('/content/drive/My Drive/finalcorpus.csv', ';', encoding= 'unicode_escape')


0    Once she asked for daddy, but went back to sle...
1    The medication they were giving her to clear u...
2        You can sit up, but you have to stay in there
3    She met his gaze defiantly for a moment longer...
4    If it's something we simply can't afford, that...
Name: text, dtype: object

In [64]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df['text'])
print(integer_encoded)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

integer_encoded[2]

[ 938 1335 1847 ... 1647 1217  330]


array([1847])

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(integer_encoded,df['label'], stratify=df['label'])

In [16]:
pip install tensorflow-text


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 54.1 MB/s eta 0:00:00


In [18]:
pip install scikeras

In [66]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text


import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from keras.layers import Dense, Dropout


In [69]:
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(units = 23, activation='relu', input_dim = X_train.shape[1]))
  model.add(tf.keras.layers.Dropout(0.1))
  model.add(tf.keras.layers.Dense(units = 8, activation='relu'))
  model.add(tf.keras.layers.Dense(units = 6, activation='relu'))
  model.add(Dense(units= 1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer=Adam(3e-4), metrics=['accuracy'])
  return model

In [70]:
model = KerasClassifier(model=create_model, verbose=0)




In [71]:
batch_size = [32, 64]
epochs = [5, 10, 15, 20, 25, 30]

param_grid = dict(batch_size = batch_size, epochs= epochs)

from sklearn.model_selection import GridSearchCV


gs=GridSearchCV(estimator=model, param_grid = param_grid, cv=5)

In [72]:
gs = gs.fit(X_train, y_train)

In [73]:

print("Best Parameters: ", gs.best_params_)
print("Best Score: ", gs.best_score_)



Best Parameters:  {'batch_size': 64, 'epochs': 25}
Best Score:  0.5113333333333333


In [74]:
best_model = gs.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy: ", test_accuracy)

Test Accuracy:  0.5
